In [ ]:
# 트랜스포머의 인코더 부분을 사용, 한국어 문장 인코딩을 위한 라이브러리 설치
!pip install -q sentence-transformers
import pandas as pd # 필요한 라이브러리 가져오기 / 판다스 라이브러리
from sentence_transformers import SentenceTransformer # 자연어 처리 라이브러리
from sklearn.metrics.pairwise import cosine_similarity # 유사도를 위한 라이브러리
# SentenceBERT - 한국어 문장 임베딩을 위한 버트 모델 가져오기
encoder = SentenceTransformer('jhgan/ko-sroberta-multitask')
# 데이터셋 가져오기
df=pd.read_csv('https://github.com/2022infotextbook/ai-basic/raw/main/2_ai_ml/qna.csv')
df.head() # 데이터 살펴보기
# 모든 데이터셋 인코딩
df['embedding'] = df['질문'].map(lambda x: list(encoder.encode(x)))
text = '요즘 성적때문에 고민이에요' # 예제 문장 입력
embedding = encoder.encode(text)
# 입력된 문장에 대한 유사도 계산
df['similarity'] = df['embedding'].map(
 lambda x: cosine_similarity([embedding], [x]).squeeze())
answer = df.loc[df['similarity'].idxmax()] # 유사도가 가장 높은 답변을 출력
#print('구분', answer['구분'])
print('유사한 질문', answer['질문'])
print('챗봇 답변', answer['답변'])
print('유사도', answer['similarity'])

유사한 질문 성적이 떨어질까봐 고민이야
챗봇 답변 네가 공부한만큼 결과 나올거야 힘내~
유사도 0.8089829683303833


In [ ]:
# gradio 라이브러리를 사용한 챗봇 만들기
# 간편한 웹앱 개발을 도와주는 gradio 라이브러리 설치, 가져오기
!pip install fastapi==0.104.1 typing_extensions==4.8.0 gradio==3.41.0
import gradio as gr
# 챗봇의 답변으로 호출될 함수 정의. (예제 문장에서 답변했던 코드 내용을 함수화)
def chatbotAnswer(user): # 사용자 입력 문장 인코딩
 embedding = encoder.encode(user)
 # 입력된 문장에 대한 유사도 계산
 df['similarity'] = df['embedding'].map(
 lambda x: cosine_similarity([embedding], [x]).squeeze())
 # 유사도가 가장 높은 답변을 구해서 answer 변수에 저장
 answer = df.loc[df['similarity'].idxmax()]
 return answer['답변']
# chatbot 앱 생성: 입력에 답변할 함수 지정 및 인터페이스 정의
chatbotApp = gr.Interface(fn=chatbotAnswer, inputs="text", outputs="text")
chatbotApp.launch(debug=True, share=True) # chatbot 앱 실행

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://e988feaa3295d62be7.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
